In [1]:
# 导入常规的数据分析库
from pyecharts import Bar,Pie
import pandas as pd
import numpy as np
import  matplotlib.pyplot as  plt
import time

In [ ]:
# 读取评论文件，取其评论列与平分列
df=pd.read_excel("all_data_meituan.xlsx")[["comment","star"]]
df.head()

In [ ]:
# 查看数据的大小
df.shape

In [ ]:
# 以30分为划分界限，大于30分的记为正类1，其余的为负类0
df['sentiment']=df['star'].apply(lambda x:1 if x>30 else 0)
df=df.drop_duplicates() ## 去掉重复的评论 
df=df.dropna()

In [ ]:
# 复制数据可以使用np.tile(arr, (3, 1))，延第一个轴的方向进行数据复制3倍。
# 如果数据不复制的话，只有1100条左右的信息，数据量太小，已使用多种算法进行拟合，效果均相差较大
X=pd.concat([df[['comment']],df[['comment']],df[['comment']]])
y=pd.concat([df.sentiment,df.sentiment,df.sentiment])
X.columns=['comment']
X.reset_index
X.shape

In [ ]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
X['cut_comment']=X["comment"].apply(chinese_word_cut)
X['cut_comment'].head()

In [ ]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42,test_size=0.25)

In [ ]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding="utf-8") as f:
        custom_stopwords_list=[i.strip() for i in f.readlines()]
    return custom_stopwords_list

In [ ]:
stop_words_file = "stopwords.txt"
stopwords = get_custom_stopwords(stop_words_file)
stopwords[-10:]

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer
vect=CountVectorizer()
vect

In [ ]:
# dir(vect)

In [ ]:
vect.fit_transform(X_train["cut_comment"])

In [ ]:
vect.fit_transform(X_train["cut_comment"]).toarray().shape

In [ ]:
# pd.DataFrame(vect.fit_transform(X_train["cut_comment"]).toarray(),columns=vect.get_feature_names()).iloc[:10,:22]
# print(vect.get_feature_names())
# #  数据维数1956，不算很大（未使用停用词）

In [ ]:
vect = CountVectorizer(token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',stop_words=frozenset(stopwords)) # 去除停用词
pd.DataFrame(vect.fit_transform(X_train['cut_comment']).toarray(), columns=vect.get_feature_names()).head()
# 1691 columns,去掉以数字为特征值的列，减少了三列编程1691 
# max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
# min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# lr=LogisticRegression(solver='saga',max_iter=10000)
lr=LogisticRegression()
pipe=make_pipeline(vect,lr)
pipe.steps

In [ ]:
pipe.fit(X_train.cut_comment, y_train)

In [ ]:
from sklearn import metrics
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

In [ ]:
metrics.confusion_matrix(y_test,y_pred)
#  准确率提高，同时降低了假阳性以及假阴性，saga的收敛的max_iter比默认的liblinear

In [ ]:
y_pred_all = pipe.predict(X['cut_comment'])
metrics.f1_score(y_true=y,y_pred=y_pred_all)
metrics.accuracy_score(y,y_pred_all)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss
# lrvc = LogisticRegressionCV()
lrvc = LogisticRegressionCV(Cs=[0.0001,0.005,0.001,0.05,0.01,0.1,0.5,1,10],scoring='accuracy',solver='saga',max_iter=10000,penalty='l2',verbose=0)
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# lr=LogisticRegression(solver='saga',max_iter =10000)
pipe=make_pipeline(vect,lrvc)
# pipe=make_pipeline(vect,lr)
# pipe.steps
pipe.fit(X_train.cut_comment, y_train)

In [ ]:
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

In [ ]:
metrics.confusion_matrix(y_test,y_pred)

In [ ]:
def get_confusion_matrix(conf,clas):
    import  matplotlib.pyplot as  plt
    fig,ax=plt.subplots(figsize=(2.5,2.5))
    ax.matshow(conf,cmap=plt.cm.Blues,alpha=0.3)
    tick_marks = np.arange(len(clas))
    plt.xticks(tick_marks,clas, rotation=45)
    plt.yticks(tick_marks, clas)
    for i in range(conf.shape[0]):
        for j in range(conf.shape[1]):
            ax.text(x=i,y=j,s=conf[i,j],
                   va='center',
                   ha='center')
    plt.xlabel("predict_label")
    plt.ylabel("true label")
    
conf=metrics.confusion_matrix(y_test,y_pred)
class_names=np.array(['0','1'])
get_confusion_matrix(np.array(conf),clas=class_names)
plt.show()